Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [3]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [4]:
pickle_file = 'notMNIST_clean.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (8970, 28, 28) (8970,)
Test set (8662, 28, 28) (8662,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [5]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (8970, 784) (8970, 10)
Test set (8662, 784) (8662, 10)


In [7]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [11]:
batch_size = 128
beta = 0.001

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, 1024]))
  biases = tf.Variable(tf.zeros([1024]))
  
  weights2 = tf.Variable(
    tf.truncated_normal([1024,num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))

  first = tf.nn.relu(tf.matmul(tf_train_dataset, weights) + biases)
  #second = tf.nn.relu(first)
  
  # Training computation.
  logits = tf.matmul(first,weights2) + biases2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  loss = loss + beta* (tf.nn.l2_loss(weights)+tf.nn.l2_loss(weights2))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights) + biases),weights2)+biases2)
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights) + biases),weights2)+biases2)

In [20]:
num_steps = 3001


with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 712.073975
Minibatch accuracy: 6.2%
Validation accuracy: 27.9%
Minibatch loss at step 500: 195.871536
Minibatch accuracy: 79.7%
Validation accuracy: 80.3%
Minibatch loss at step 1000: 116.784584
Minibatch accuracy: 77.3%
Validation accuracy: 80.6%
Minibatch loss at step 1500: 68.633492
Minibatch accuracy: 84.4%
Validation accuracy: 79.9%
Minibatch loss at step 2000: 41.820942
Minibatch accuracy: 84.4%
Validation accuracy: 83.0%
Minibatch loss at step 2500: 25.239935
Minibatch accuracy: 85.2%
Validation accuracy: 84.9%
Minibatch loss at step 3000: 15.409193
Minibatch accuracy: 87.5%
Validation accuracy: 86.2%
Test accuracy: 92.2%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [21]:
num_steps = 3001
train_label_over = train_labels[1:300,:]
train_dataset_over = train_dataset[1:300,:]

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_label_over.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset_over[offset:(offset + batch_size), :]
    batch_labels = train_label_over[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 626.389648
Minibatch accuracy: 10.9%
Validation accuracy: 33.2%
Minibatch loss at step 500: 190.518250
Minibatch accuracy: 100.0%
Validation accuracy: 69.6%
Minibatch loss at step 1000: 115.540710
Minibatch accuracy: 100.0%
Validation accuracy: 69.6%
Minibatch loss at step 1500: 70.070213
Minibatch accuracy: 100.0%
Validation accuracy: 69.7%
Minibatch loss at step 2000: 42.494431
Minibatch accuracy: 100.0%
Validation accuracy: 69.7%
Minibatch loss at step 2500: 25.770988
Minibatch accuracy: 100.0%
Validation accuracy: 69.7%
Minibatch loss at step 3000: 15.629160
Minibatch accuracy: 100.0%
Validation accuracy: 69.9%
Test accuracy: 77.5%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [30]:
batch_size = 128
beta = 0.001
keep_value = 0.5

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, 1024]))
  biases = tf.Variable(tf.zeros([1024]))
  
  weights2 = tf.Variable(
    tf.truncated_normal([1024,num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))

  first = tf.nn.dropout(tf.nn.relu(tf.matmul(tf_train_dataset, weights) + biases),keep_value)
  #second = tf.nn.relu(first)
  
  # Training computation.
  logits = tf.matmul(first,weights2) + biases2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  loss = loss + beta* (tf.nn.l2_loss(weights)+tf.nn.l2_loss(weights2))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.dropout(tf.nn.relu(tf.matmul(tf_valid_dataset, weights) + biases),keep_value),weights2)+biases2)
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights) + biases),weights2)+biases2)

In [31]:
num_steps = 10001


with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 937.390869
Minibatch accuracy: 10.9%
Validation accuracy: 29.4%
Minibatch loss at step 500: 228.439499
Minibatch accuracy: 64.1%
Validation accuracy: 68.4%
Minibatch loss at step 1000: 134.754333
Minibatch accuracy: 67.2%
Validation accuracy: 65.0%
Minibatch loss at step 1500: 73.904327
Minibatch accuracy: 71.1%
Validation accuracy: 67.2%
Minibatch loss at step 2000: 44.780346
Minibatch accuracy: 74.2%
Validation accuracy: 71.6%
Minibatch loss at step 2500: 26.069742
Minibatch accuracy: 75.0%
Validation accuracy: 74.1%
Minibatch loss at step 3000: 15.968667
Minibatch accuracy: 78.1%
Validation accuracy: 76.8%
Minibatch loss at step 3500: 9.933156
Minibatch accuracy: 81.2%
Validation accuracy: 78.4%
Minibatch loss at step 4000: 6.285362
Minibatch accuracy: 82.8%
Validation accuracy: 78.7%
Minibatch loss at step 4500: 4.014479
Minibatch accuracy: 81.2%
Validation accuracy: 79.6%
Minibatch loss at step 5000: 2.767386
Minibatch accuracy: 82.0%
Validati

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [10]:
from numpy.ma import sqrt

batch_size = 128
beta = 0.0001
keep_value = 0.5


def buildTrainLayerWithDropout(w, b, dataset):
    f1 = tf.matmul(dataset,w)+b
    return tf.nn.dropout(tf.nn.relu(f1), keep_value)

def buildTestLayerWithoutDropout(w, b,dataset):
    return tf.nn.relu(tf.matmul(dataset,w)+b)


def calcOptimalStd(numInput):
    return sqrt(2.0/numInput)

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  
  w1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, 1024],0,calcOptimalStd(image_size*image_size)))
  b1 = tf.Variable(tf.zeros([1024]))
    
  h1 = buildTrainLayerWithDropout(w1,b1,tf_train_dataset)

  w2 = tf.Variable(
    tf.truncated_normal([1024, 512], 0,calcOptimalStd(1024)))
  b2 = tf.Variable(tf.zeros([512]))

  h2 =  buildTrainLayerWithDropout(w2,b2,h1)
    
  
  w3 = tf.Variable(
    tf.truncated_normal([512, 256], 0,calcOptimalStd(512)))
  b3 = tf.Variable(tf.zeros([256]))

  h3 =  buildTrainLayerWithDropout(w3,b3,h2)
  
  
  weights = tf.Variable(
    tf.truncated_normal([256,num_labels], 0,calcOptimalStd(256)))
  biases = tf.Variable(tf.zeros([num_labels]))

  #first = tf.nn.dropout(tf.nn.relu(tf.matmul(tf_train_dataset, weights) + biases),keep_value)
  #second = tf.nn.relu(first)
  
  # Training computation.
  logits = tf.matmul(h3,weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  loss = loss + beta* (tf.nn.l2_loss(weights)+tf.nn.l2_loss(w1)+tf.nn.l2_loss(w2)+tf.nn.l2_loss(w3))
  
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.5, global_step, 1,0.9999)  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss,global_step = global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)

  v1 = buildTrainLayerWithDropout(w1,b1,tf_valid_dataset)
  v2 = buildTrainLayerWithDropout(w2,b2,v1)
  v3 = buildTrainLayerWithDropout(w3,b3,v2)
  valid_prediction = tf.nn.softmax(
    tf.matmul(v3,weights) + biases)

  t1 = buildTestLayerWithoutDropout(w1,b1,tf_test_dataset)
  t2 = buildTestLayerWithoutDropout(w2,b2,t1)
  t3 = buildTestLayerWithoutDropout(w3,b3,t2)
    
  test_prediction = tf.nn.softmax(tf.matmul(t3,weights) + biases)

In [ ]:
num_steps = 20000


with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.766420
Minibatch accuracy: 15.6%
Validation accuracy: 16.4%
Minibatch loss at step 500: 0.823986
Minibatch accuracy: 76.6%
Validation accuracy: 81.1%
Minibatch loss at step 1000: 0.689377
Minibatch accuracy: 83.6%
Validation accuracy: 82.2%
Minibatch loss at step 1500: 0.622962
Minibatch accuracy: 85.9%
Validation accuracy: 83.7%
Minibatch loss at step 2000: 0.592396
Minibatch accuracy: 85.2%
Validation accuracy: 84.1%
Minibatch loss at step 2500: 0.775990
Minibatch accuracy: 82.0%
Validation accuracy: 84.5%
Minibatch loss at step 3000: 0.592042
Minibatch accuracy: 85.9%
Validation accuracy: 85.0%
Minibatch loss at step 3500: 0.595238
Minibatch accuracy: 85.9%
Validation accuracy: 85.0%
Minibatch loss at step 4000: 0.511894
Minibatch accuracy: 87.5%
Validation accuracy: 85.7%
Minibatch loss at step 4500: 0.447323
Minibatch accuracy: 90.6%
Validation accuracy: 85.9%
Minibatch loss at step 5000: 0.502847
Minibatch accuracy: 89.8%
Validation accurac